In [1]:
from tensorflow import keras

In [3]:
from keras.models import Sequential
from keras.layers import Dense,Activation,Dropout
from keras.preprocessing import text,sequence
from keras import utils

In [11]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
%matplotlib notebook
import seaborn as sns
sns.set(style="ticks", color_codes=True)

In [53]:
train_json = 'C:\\Users\\Gracy\\Python Courses\\Python_ML_Projects\\Text-Mining-Whats-cooking\\whats-cooking\\train.json'
data_train = pd.read_json(train_json,orient = 'records')
test_json = 'C:\\Users\\Gracy\\Python Courses\\Python_ML_Projects\\Text-Mining-Whats-cooking\\whats-cooking\\test.json'
data_test = pd.read_json(test_json, orient = 'records')
data_train['ingredients'] = data_train.apply(lambda row: ','.join(row['ingredients']), axis=1)
data_train_ing = data_train['ingredients'].str.split(',\s*', expand=True)
#data_train_ing = data_train_ing.add_prefix('ing')

In [54]:
print(data_train_ing.info())
print(data_train.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39774 entries, 0 to 39773
Data columns (total 65 columns):
0     39774 non-null object
1     39752 non-null object
2     39559 non-null object
3     39012 non-null object
4     37899 non-null object
5     36024 non-null object
6     33365 non-null object
7     30045 non-null object
8     26505 non-null object
9     22747 non-null object
10    19072 non-null object
11    15555 non-null object
12    12412 non-null object
13    9721 non-null object
14    7433 non-null object
15    5623 non-null object
16    4182 non-null object
17    3028 non-null object
18    2125 non-null object
19    1522 non-null object
20    1008 non-null object
21    695 non-null object
22    477 non-null object
23    334 non-null object
24    243 non-null object
25    169 non-null object
26    123 non-null object
27    103 non-null object
28    76 non-null object
29    55 non-null object
30    40 non-null object
31    29 non-null object
32    25 non-null object
33  

In [60]:
train_data = data_train.join(data_train_ing)
train_data = train_data.drop(['id','ingredients'],axis=1)
train_data

,cuisine,0,1,2,3,4,5,6,7,8,...,55,56,57,58,59,60,61,62,63,64
0,greek,romaine lettuce,black olives,grape tomatoes,garlic,pepper,purple onion,seasoning,garbanzo beans,feta cheese crumbles,...,None,None,None,None,None,None,None,None,None,None
1,southern_us,plain flour,ground pepper,salt,tomatoes,ground black pepper,thyme,eggs,green tomatoes,yellow corn meal,...,None,None,None,None,None,None,None,None,None,None
2,filipino,eggs,pepper,salt,mayonaise,cooking oil,green chilies,grilled chicken breasts,garlic powder,yellow onion,...,None,None,None,None,None,None,None,None,None,None
3,indian,water,vegetable oil,wheat,salt,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
4,indian,black pepper,shallots,cornflour,cayenne pepper,onions,garlic paste,milk,butter,salt,...,None,None,None,None,None,None,None,None,None,None
5,jamaican,plain flour,sugar,butter,eggs,fresh ginger root,salt,ground cinnamon,milk,vanilla extract,...,None,None,None,None,None,None,None,None,None,None
6,spanish,olive oil,salt,medium shrimp,pepper,garlic,chopped cilantro,jalapeno chilies,flat leaf parsley,skirt steak,...,None,None,None,None,None,None,None,None,None,None
7,italian,sugar,pistachio nuts,white almond bark,flour,vanilla extract,olive oil,almond extract,eggs,baking powder,...,None,None,None,None,None,None,None,None,None,None
8,mexican,olive oil,purple onion,fresh pineapple,pork,poblano peppers,corn tortillas,cheddar cheese,ground black pepper,salt,...,None,None,None,None,None,None,None,None,None,None
9,italian,chopped tomatoes,fresh basil,garlic,extra-virgin olive oil,kosher salt,flat leaf parsley,None,None,None,...,None,None,None,None,None,None,None,None,None,None


In [61]:
X = train_data.iloc[:,1:]
y = train_data['cuisine']
print(X.shape)
print(y.shape)
X.head()

(39774, 65)
(39774,)


,0,1,2,3,4,5,6,7,8,9,...,55,56,57,58,59,60,61,62,63,64
0,romaine lettuce,black olives,grape tomatoes,garlic,pepper,purple onion,seasoning,garbanzo beans,feta cheese crumbles,None,...,None,None,None,None,None,None,None,None,None,None
1,plain flour,ground pepper,salt,tomatoes,ground black pepper,thyme,eggs,green tomatoes,yellow corn meal,milk,...,None,None,None,None,None,None,None,None,None,None
2,eggs,pepper,salt,mayonaise,cooking oil,green chilies,grilled chicken breasts,garlic powder,yellow onion,soy sauce,...,None,None,None,None,None,None,None,None,None,None
3,water,vegetable oil,wheat,salt,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
4,black pepper,shallots,cornflour,cayenne pepper,onions,garlic paste,milk,butter,salt,lemon juice,...,None,None,None,None,None,None,None,None,None,None


In [62]:
num_classes = y.unique().size

In [63]:
# Avoid previous column names appending
X1 = pd.get_dummies(X,prefix='',prefix_sep='')
X1.head()

,( oz.) tomato sauce,(14.5 oz.) diced tomatoes,1% low-fat buttermilk,1% low-fat cottage cheese,1% low-fat milk,2% reduced-fat milk,40% less sodium taco seasoning,Accent Seasoning,Alfredo sauce,American cheese,...,butter,panko breadcrumbs,low-fat milk,multi-grain penne pasta,extra-virgin olive oil,cilantro leaves,green pepper,shredded mozzarella cheese,fresh parsley,spaghetti
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [64]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X1,y,test_size=0.2,random_state=123)

In [69]:
y_train

22321         chinese
2758          mexican
23666         italian
9483             thai
32137         mexican
34860          french
25703         italian
34436        moroccan
32521     southern_us
19760     southern_us
25535         chinese
11880          indian
7132          mexican
8357          chinese
30060        japanese
26267     southern_us
35495         italian
18822          french
27469         british
31670         chinese
15771        filipino
5346          italian
9845          italian
16158     southern_us
6868          italian
19039         italian
849            french
29036           greek
19393         mexican
31269         italian
             ...     
2428           french
4195         filipino
17543         mexican
23728          indian
22461            thai
6648          italian
30255          indian
39771           irish
6484          italian
26626         mexican
7985         filipino
17476         chinese
14944     southern_us
31140     southern_us
3325      

In [66]:
input_dim = X_train.shape[1]
print(input_dim)
num_classes = y.unique().size
print(num_classes)

44075
20


In [70]:
from sklearn.preprocessing import LabelBinarizer, LabelEncoder
encoder = LabelEncoder()
encoder.fit(y_train)
y_train = encoder.transform(y_train)
y_test = encoder.transform(y_test)


In [71]:
y_train

array([ 3, 13,  9, ..., 17, 13,  9])

In [ ]:
input_dim = X_train.shape[1]
num_classes = y.unique().size
model = Sequential()
model.add(Dense(512, input_shape=(input_dim,)))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(output_dim=num_classes))
model.add(Activation('softmax'))

model.compile(loss='sparse_categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
              
#history = model.fit(X_train, y_train,batch_size=2711,epochs=2,verbose=1,validation_split=0.1)
history = model.fit(X_train, y_train,epochs=2,verbose=1,validation_split=0.1)

C:\Users\Gracy\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:7: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(units=20)`
  import sys


Train on 28637 samples, validate on 3182 samples
Epoch 1/2
   64/28637 [..............................] - ETA: 2:19:05 - loss: 2.9913 - acc: 0.0469    

C:\Users\Gracy\Anaconda3\envs\tensorflow\lib\site-packages\keras\callbacks.py:122: UserWarning: Method on_batch_end() is slow compared to the batch update (1.538911). Check your callbacks.
  % delta_t_median)


28637/28637 [==============================] - 396s 14ms/step - loss: 1.5518 - acc: 0.5617 - val_loss: 1.1267 - val_acc: 0.6675
Epoch 2/2
25760/28637 [=========================>....] - ETA: 36s - loss: 0.5539 - acc: 0.8435

In [ ]:
#score = model.evaluate(x_test, y_test,batch_size=batch_size, verbose=1)
score = model.evaluate(X_test, y_test, verbose=1)
print('Test accuracy:', score[1])